In [1]:
import findspark
findspark.find()
findspark.init(edit_profile=True)
import pyspark
sc = pyspark.SparkContext()

In [102]:
from functools import partial
import time

In [3]:
def generatePairs(node_set):
    edges = []
    for i in node_set:
        for j in node_set:
            if(i != j):
                edges.append((i, j))
    return edges

In [4]:
rawData = sc.textFile("source.csv").map(lambda line: line.split('","')).map(lambda x: [x[0]+'"', '"'+x[1]]).cache()
charactersRDD = rawData.map(lambda x: x[0]).distinct()
graphRDD = charactersRDD.map(lambda x: (x, []))
edges_tmp = rawData.map(lambda x: (x[1], x[0])).groupByKey().map(lambda x: list(x[1]))

In [5]:
rawData.take(5)

[[u'"FROST, CARMILLA"', u'"AA2 35"'],
 [u'"KILLRAVEN/JONATHAN R"', u'"AA2 35"'],
 [u'"M\'SHULLA"', u'"AA2 35"'],
 [u'"24-HOUR MAN/EMMANUEL"', u'"AA2 35"'],
 [u'"OLD SKULL"', u'"AA2 35"']]

In [6]:
##############################################
# Generate Graph
# Problems: 
# 1. groupByKey so inefficient, replace!
# 2. some isolated characters (nodes) failed to be added in the graph (see the following cell)
##############################################

MC_Graph = edges_tmp.flatMap(generatePairs).groupByKey().mapValues(lambda x: list(set(x)))
connected_nodes = MC_Graph.keys().collect()
isolated_nodes = charactersRDD.filter(lambda K: K not in connected_nodes).map(lambda x: (x, []))
MC_Graph = MC_Graph.union(isolated_nodes).cache()


In [38]:
# For testing
print MC_Graph.count()
print charactersRDD.count()
print MC_Graph.filter(lambda (K, V): V == []).collect() 
MC_Graph.take(10)

6444
6444
[(u'"KULL"', []), (u'"RUNE"', []), (u'"LUNATIK II"', []), (u'"GERVASE, LADY ALYSSA"', []), (u'"RED WOLF II"', []), (u'"ZANTOR"', []), (u'"CLUMSY FOULUP"', []), (u'"JOHNSON, LYNDON BAIN"', []), (u'"MARVEL BOY II/MARTIN"', []), (u'"BERSERKER II"', []), (u'"RANDAK"', []), (u'"SHARKSKIN"', []), (u'"CALLAHAN, DANNY"', []), (u'"GIURESCU, RADU"', []), (u'"BLARE/"', []), (u'"SEA LEOPARD"', []), (u'"FENRIS"', []), (u'"DEATHCHARGE"', [])]


[(u'"VIGIL"', [u'"THING/BENJAMIN J. GR"']),
 (u'"TOWER"',
  [u'"BLOB/FRED J. DUKES"',
   u'"CANNONBALL II/SAM GU"',
   u'"GIDEON"',
   u'"AVALANCHE/DOMINIC PE"',
   u'"CYCLOPS/SCOTT SUMMER"',
   u'"DESTINY II/IRENE ADL"',
   u'"HODGE, CAMERON"',
   u'"BOOMER/TABITHA SMITH"',
   u'"SPIRAL"',
   u'"ROSEN, WALTER"',
   u'"NAMORITA/NITA PRENTI"',
   u'"WIZ KID/TAKESHI MATS"',
   u'"FURMINTZ, HARMON"',
   u'"JUSTICE II/VANCE AST"',
   u'"SKIDS/SALLY BLEVINS"',
   u'"SOUTHERN, CANDY"',
   u'"CANTOR, VERA"',
   u'"COPYCAT/VANESSA"',
   u'"MADDICKS, PROFESSOR "',
   u'"HARNESS/ERIKA BENSON"',
   u'"ANGEL/WARREN KENNETH"',
   u'"NOVA/RICHARD RIDER"',
   u'"CRIMSON COMMANDO/FRA"',
   u'"SHATTERSTAR II/GAVEE"',
   u'"TIMESHADOW"',
   u'"FIREFIST/RUSTY COLLI"',
   u'"MADDICKS, ARTHUR ART"',
   u'"CARGIL/JOANNA CARGIL"',
   u'"FIRESTAR/ANGELICA JO"',
   u'"THUNDERBIRD II/JAMES"',
   u'"BEAST/HENRY &HANK& P"',
   u'"LEECH"',
   u'"SPEEDBALL/ROBBIE BAL"',
   u'"APOCALYPSE/EN SABAH "',
   u'"TOAD/MORT

In [8]:
a = sc.parallelize([(1,2), (3,4)])
dict(a.collect())

{1: 2, 3: 4}

In [109]:
def BFS(MC_Graph, init_node):
    MC_Graph_bfs = MC_Graph.map(lambda (x, y): (x, (y, 0, False)) 
                                if (x == init_node) 
                                else (x, (y, float("inf"), False)))
    tmp = dict(MC_Graph_bfs.collect())
    broadcast_graph = sc.broadcast(tmp)
    
    diameter = 10
    for i in xrange(diameter):    
        updated_nodes = MC_Graph_bfs.filter(lambda x: x[1][2] == False).flatMap(partial(update_graph, mc_graph = broadcast_graph)).groupByKey()
        updated_nodes = updated_nodes.mapValues(reduce_nodes)
        
        print 'activation number: ', updated_nodes.count()
        
        MC_Graph_bfs = MC_Graph_bfs.filter(lambda x: x[1][2] == True).union(updated_nodes)
        
    touched_nodes_num = MC_Graph_bfs.filter(lambda x: x[1][1] < float("inf")).count()

    return touched_nodes_num

def update_graph(node, mc_graph):
    # extract values from node for clarity
    character_name = node[0]
    adj_list = node[1][0]
    dist = node[1][1]
    is_used = node[1][2]
    
    mc_graph = mc_graph.value
    result = []
    if dist < float("inf"):
        for i in adj_list:
            new_node = (i, mc_graph[i])
            if new_node[1][1] > (dist + 1):
                new_node[1][1] = (dist + 1)
                result.append(new_node)

        node[1][2] = True
    
    result.append(node)
    return result

def reduce_nodes(nodes_list):
    nodes_list = list(nodes_list)
    return sorted(nodes_list, key = lambda x: x[1])[0]

In [97]:
a = [('a',(1,1)), ('a',(3,1)), ('a',(2,1))]


In [98]:
sorted(a, key = lambda x: x[1][0])[0]

('a', (1, 1))

In [99]:
a = sc.parallelize([('a',(1,1)), ('a',(2,1)), ('a',(3,1))])

In [100]:
a.sortBy(lambda x: x[1][1]).collect()

[('a', (1, 1)), ('a', (2, 1)), ('a', (3, 1))]

In [110]:
start_time = time.time()
result = BFS(MC_Graph, 'MISS THING/MARY')
print time.time() - start_time


activation number:  6444
activation number:  6444
activation number:  6444
activation number:  6444
activation number:  6444
activation number:  6444
activation number:  6444
activation number:  6444
activation number:  6444
activation number:  6444
422.994869947


In [112]:
print result


0
